In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
porcesseddir = '/home/mei/nas/docker/processedData/'

In [2]:
def round_up(x, base=5):
    return base * round(x/base)

In [5]:
df = pd.read_csv(datadir + 'diagnosis.csv.gz', compression='gzip')

In [6]:
df.head()

,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,4222318,141168,False,72,cardiovascular|chest pain / ASHD|coronary arte...,"414.00, I25.10",Other
1,3370568,141168,True,118,cardiovascular|ventricular disorders|cardiomyo...,NaN,Other
2,4160941,141168,False,72,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
3,4103261,141168,True,118,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
4,3545241,141168,True,118,cardiovascular|ventricular disorders|congestiv...,"428.0, I50.9",Other


In [7]:
df_patient = pd.read_csv(porcesseddir + 'selected_patients.csv')

In [9]:
df_selected_dx = pd.merge(df_patient, df, on='patientunitstayid')

In [11]:
df = df_selected_dx.copy()
df = df.set_index(['patientunitstayid', 'diagnosisoffset'])
df = df.drop(df.index[np.where(df.index.get_level_values('diagnosisoffset') < 0)])
df = df.drop(columns = ['diagnosisid'])
df.sort_index(level = ['patientunitstayid', 'diagnosisoffset'], inplace = True)
# Resample every 5 mins
df.rename(round_up, level = 'diagnosisoffset', inplace = True)

In [12]:
df.head( )

hospitaladmitoffset  gender age  \
patientunitstayid diagnosisoffset                                    
141168            70                                 0  Female  70   
                  70                                 0  Female  70   
                  70                                 0  Female  70   
                  70                                 0  Female  70   
                  70                                 0  Female  70   

                                                               apacheadmissiondx  \
patientunitstayid diagnosisoffset                                                  
141168            70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   

                                  unitadmitsource  admissionweight  \
patientunitstayid diagnosisoffset                                    
141168            70                 Direct Admit             84.3   
                  70                 Direct Admit             84.3   
                  70                 Direct Admit             84.3   
                  70                 Direct Admit             84.3   
                  70                 Direct Admit             84.3   

                                   dischargeweight unitdischargelocation  \
patientunitstayid diagnosisoffset                                          
141168            70                          85.8                 Death   
                  70                          85.8                 Death   
                  70                          85.8                 Death   
                  70                          85.8                 Death   
                  70                          85.8                 Death   

                                  unitdischargestatus  activeupondischarge  \
patientunitstayid diagnosisoffset                                            
141168            70                          Expired                False   
                  70                          Expired                False   
                  70                          Expired                False   
                  70                          Expired                False   
                  70                          Expired                False   

                                                                     diagnosisstring  \
patientunitstayid diagnosisoffset                                                      
141168            70               cardiovascular|chest pain / ASHD|coronary arte...   
                  70                         pulmonary|disorders of the airways|COPD   
                  70               cardiovascular|ventricular disorders|congestiv...   
                  70               cardiovascular|ventricular disorders|cardiomyo...   
                  70                 renal|disorder of kidney|chronic kidney disease   

                                         icd9code diagnosispriority  
patientunitstayid diagnosisoffset                                    
141168            70               414.00, I25.10             Other  
                  70                491.20, J44.9             Other  
                  70                 428.0, I50.9             Other  
                  70                          NaN             Other  
                  70                 585.9, N18.9             Other

In [13]:
# df.to_csv(porcesseddir + 'selected_diagnosis.csv',  index=True)
# print("Data exported successfully to 'selected_diagnosis.csv'.")


Data exported successfully to 'selected_diagnosis.csv'.


In [14]:
selected_diag_p=len(list(df.index.get_level_values('patientunitstayid').unique()))
selected_diag_x=len(df)
print("There are {} unique patientunitstayid in the selected patients with the total {} records.".format(selected_diag_p,selected_diag_x))

There are 95667 unique patientunitstayid in the selected patients with the total 1393336 records.


In [15]:
df1 = df[['diagnosisstring', 'icd9code']]
df1 = df1.reset_index()
df1['diagnosisstring'] = df1['diagnosisstring'].str.replace('|', '/')
split = df1['diagnosisstring'].str.split('/')
df1['first'] = split.str[0]
df1['second'] = split.str[1]
df1['third'] = split.str[2]
df1['fourth'] = split.str[3]
df1['fifth'] = split.str[4]
df1['sixth'] = split.str[5]
df1['seventh'] = split.str[6]
df1.head()

,patientunitstayid,diagnosisoffset,diagnosisstring,icd9code,first,second,third,fourth,fifth,sixth,seventh
0,141168,70,cardiovascular/chest pain / ASHD/coronary arte...,"414.00, I25.10",cardiovascular,chest pain,ASHD,coronary artery disease,known,NaN,NaN
1,141168,70,pulmonary/disorders of the airways/COPD,"491.20, J44.9",pulmonary,disorders of the airways,COPD,NaN,NaN,NaN,NaN
2,141168,70,cardiovascular/ventricular disorders/congestiv...,"428.0, I50.9",cardiovascular,ventricular disorders,congestive heart failure,NaN,NaN,NaN,NaN
3,141168,70,cardiovascular/ventricular disorders/cardiomyo...,NaN,cardiovascular,ventricular disorders,cardiomyopathy,NaN,NaN,NaN,NaN
4,141168,70,renal/disorder of kidney/chronic kidney disease,"585.9, N18.9",renal,disorder of kidney,chronic kidney disease,NaN,NaN,NaN,NaN


In [16]:
frequency1 = pd.DataFrame()
frequency1['noAnnotations'] = df1['first'].value_counts() 
frequency1['%'] = df1['first'].value_counts() / len(df1.index) * 100
frequency1

,noAnnotations,%
first,,
cardiovascular,348062,24.980479
pulmonary,241767,17.351665
neurologic,181764,13.045238
renal,165398,11.870647
gastrointestinal,107816,7.737976
endocrine,101807,7.306709
infectious diseases,83485,5.991735
hematology,67168,4.820661
burns,49800,3.574156


In [17]:
from bigtree import Node, list_to_tree, tree_to_dot, dataframe_to_tree, tree_to_pillow

for disease in frequency1.index.get_level_values('first'):
    df_ = df1[df1['first'] == disease].copy()
    df_.set_index(['patientunitstayid', 'diagnosisoffset'], inplace = True)
    df_['diagnosis'] = df_['first'] + '/' + df_['second'] + '/' + df_['third']
    
    paths_mult_ = df_.groupby('diagnosis', dropna = True)['diagnosis'].size().to_frame('noAnnotations')
    paths_mult_.reset_index(inplace=True)
    paths_mult_.sort_values('noAnnotations', ascending = False, inplace = True)

    paths_ = list(pd.unique(df_['diagnosis']))
    root_ = dataframe_to_tree(paths_mult_, sep = '/')
    root_.show(attr_list=["noAnnotations"])
    pillow_image_ = tree_to_pillow(root_)

cardiovascular
├── shock 
│   └──  hypotension [noAnnotations=103207]
├── chest pain 
│   └──  ASHD [noAnnotations=62269]
├── ventricular disorders
│   ├── congestive heart failure [noAnnotations=30103]
│   ├── hypertension [noAnnotations=16160]
│   ├── cardiomyopathy [noAnnotations=6522]
│   ├── acute pulmonary edema [noAnnotations=5339]
│   ├── low cardiac output state [noAnnotations=634]
│   ├── cardiac mural thrombus [noAnnotations=176]
│   ├── cardiac injury, blunt [noAnnotations=4]
│   └── cardiac injury, penetrating [noAnnotations=2]
├── arrhythmias
│   ├── atrial fibrillation [noAnnotations=29102]
│   ├── sinus tachycardia [noAnnotations=6582]
│   ├── bradycardia [noAnnotations=6471]
│   ├── ventricular tachycardia [noAnnotations=3915]
│   ├── AV block [noAnnotations=2518]
│   ├── atrial flutter [noAnnotations=2189]
│   ├── SVT [noAnnotations=1930]
│   ├── syncope [noAnnotations=1439]
│   ├── ventricular fibrillation [noAnnotations=729]
│   ├── pacemaker [noAnnotations=350]
│  

In [18]:
cardio = df1[df1['first'] == 'cardiovascular']
cardio.set_index(['patientunitstayid', 'diagnosisoffset'], inplace = True)


In [19]:

cardiofreq = pd.DataFrame()
cardiofreq['noAnnotations'] = cardio['second'].value_counts() 
cardiofreq['%'] = round(cardio['second'].value_counts() / len(cardio.index) * 100, 2)
cardiofreq

,noAnnotations,%
second,,
shock,103207,29.65
chest pain,62269,17.89
ventricular disorders,58940,16.93
arrhythmias,55732,16.01
vascular disorders,36662,10.53
cardiac arrest,18758,5.39
valvular disease,3626,1.04
cardiac surgery,2799,0.80
diseases of the aorta,2306,0.66


In [20]:

cardiofreq = pd.DataFrame()
cardiofreq['noAnnotations'] = cardio['third'].value_counts() 
cardiofreq['%'] = round(cardio['third'].value_counts() / len(cardio.index) * 100, 2)
cardiofreq

,noAnnotations,%
third,,
hypotension,103207,29.65
ASHD,62269,17.89
hypertension,44626,12.82
congestive heart failure,30103,8.65
atrial fibrillation,29102,8.36
cardiac arrest,18758,5.39
sinus tachycardia,6582,1.89
cardiomyopathy,6522,1.87
bradycardia,6471,1.86


In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import seaborn as sns
# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline

In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
from sqlalchemy import create_engine
con= create_engine('postgresql://eicu@localhost:5432/eicu')

In [4]:
def round_up(x, base=5):
    return base * round(x/base)

In [5]:
query = query_schema + """
select *
from diagnosis

"""

df_addx = pd.read_sql_query(query, con)
n_dx = df_addx['patientunitstayid'].nunique()
print("There are {} icu patients in the diagnosis record.".format(n_dx))

There are 173109 icu patients in the diagnosis record.


In [23]:
query = query_schema + """

with selected_patients as (
SELECT DISTINCT patientunitstayid,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,	dischargeweight,unitdischargelocation,	unitdischargestatus
FROM patient
WHERE unitadmitsource IN ('Emergency Department', 'Direct Admit') 
)
select dx.diagnosisid,dx.diagnosisoffset, dx.activeupondischarge,dx.diagnosisstring,dx.	icd9code, dx.diagnosispriority,sp.*
from diagnosis dx
join selected_patients sp
  on dx.patientunitstayid = sp.patientunitstayid
 """

df_diagnosis_selected = pd.read_sql_query(query, con)
df_diagnosis_selected.head()

,diagnosisid,diagnosisoffset,activeupondischarge,diagnosisstring,icd9code,diagnosispriority,patientunitstayid,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,4222318,72,False,cardiovascular|chest pain / ASHD|coronary arte...,"414.00, I25.10",Other,141168,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
1,3370568,118,True,cardiovascular|ventricular disorders|cardiomyo...,,Other,141168,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
2,4160941,72,False,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other,141168,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
3,4103261,118,True,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other,141168,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired
4,3545241,118,True,cardiovascular|ventricular disorders|congestiv...,"428.0, I50.9",Other,141168,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired


In [24]:
# df_diagnosis_selected.to_csv('/Users/meisun/Documents/study/thesis/master project/data-preprocessing/selected_patients_diagnosis.csv')

In [8]:
df_diagnosis_selected = pd.read_csv('/Users/meisun/Documents/study/thesis/master project/data-preprocessing/selected_patients_diagnosis.csv')

In [29]:
#  data to be grouped by patients (patientunitstayid) and their specific diagnosis IDs (admissiondxid),
df= df_diagnosis_selected.copy()
df= df.set_index(['patientunitstayid', 'diagnosisoffset'])
df = df.drop(df.index[np.where(df.index.get_level_values('diagnosisoffset') < 0)])

df.sort_index(level = ['patientunitstayid', 'diagnosisoffset'], inplace = True)
df= df[['activeupondischarge', 'diagnosisstring','icd9code', 'diagnosispriority','gender','age','apacheadmissiondx','unitadmitsource','admissionweight',	'dischargeweight','unitdischargelocation',	'unitdischargestatus']]
# Resample every 5 mins
df.rename(round_up, level = 'diagnosisoffset', inplace = True)


In [30]:
df.head(n=10)

activeupondischarge  \
patientunitstayid diagnosisoffset                       
141168            70                            False   
                  70                            False   
                  70                            False   
                  70                            False   
                  70                            False   
                  70                            False   
                  120                            True   
                  120                            True   
                  120                            True   
                  120                            True   

                                                                     diagnosisstring  \
patientunitstayid diagnosisoffset                                                      
141168            70               cardiovascular|chest pain / ASHD|coronary arte...   
                  70                         pulmonary|disorders of the airways|COPD   
                  70               cardiovascular|ventricular disorders|congestiv...   
                  70               cardiovascular|ventricular disorders|cardiomyo...   
                  70                 renal|disorder of kidney|chronic kidney disease   
                  70               cardiovascular|arrhythmias|atrial fibrillation...   
                  120              cardiovascular|ventricular disorders|cardiomyo...   
                  120                        pulmonary|disorders of the airways|COPD   
                  120              cardiovascular|ventricular disorders|congestiv...   
                  120              cardiovascular|arrhythmias|atrial fibrillation...   

                                         icd9code diagnosispriority  gender  \
patientunitstayid diagnosisoffset                                             
141168            70               414.00, I25.10             Other  Female   
                  70                491.20, J44.9             Other  Female   
                  70                 428.0, I50.9             Other  Female   
                  70                                          Other  Female   
                  70                 585.9, N18.9             Other  Female   
                  70                427.31, I48.0           Primary  Female   
                  120                                         Other  Female   
                  120               491.20, J44.9             Other  Female   
                  120                428.0, I50.9             Other  Female   
                  120               427.31, I48.0           Primary  Female   

                                  age  \
patientunitstayid diagnosisoffset       
141168            70               70   
                  70               70   
                  70               70   
                  70               70   
                  70               70   
                  70               70   
                  120              70   
                  120              70   
                  120              70   
                  120              70   

                                                               apacheadmissiondx  \
patientunitstayid diagnosisoffset                                                  
141168            70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  70               Rhythm disturbance (atrial, supraventricular)   
                  120              Rhythm disturbance (atrial, supraventricular)   
                  120              Rhythm disturbance (atrial, supraventricular)   
          

In [31]:
n_diagnosis_selected= len(df.index)
print("There are {} selected patients in the diagnosis table with total {} records.".format(len(list(df.index.get_level_values('patientunitstayid').unique())),n_diagnosis_selected))

There are 95667 selected patients in the diagnosis table with total 1393336 records.


In [32]:
df1 = df[['diagnosisstring', 'icd9code']]
df1 = df1.reset_index()
df1['diagnosisstring'] = df1['diagnosisstring'].str.replace('|', '/')
split = df1['diagnosisstring'].str.split('/')
df1['first'] = split.str[0]
df1['second'] = split.str[1]
df1['third'] = split.str[2]
df1['fourth'] = split.str[3]
df1['fifth'] = split.str[4]
df1['sixth'] = split.str[5]
df1['seventh'] = split.str[6]
df1.head()

,patientunitstayid,diagnosisoffset,diagnosisstring,icd9code,first,second,third,fourth,fifth,sixth,seventh
0,141168,70,cardiovascular/chest pain / ASHD/coronary arte...,"414.00, I25.10",cardiovascular,chest pain,ASHD,coronary artery disease,known,NaN,NaN
1,141168,70,pulmonary/disorders of the airways/COPD,"491.20, J44.9",pulmonary,disorders of the airways,COPD,NaN,NaN,NaN,NaN
2,141168,70,cardiovascular/ventricular disorders/congestiv...,"428.0, I50.9",cardiovascular,ventricular disorders,congestive heart failure,NaN,NaN,NaN,NaN
3,141168,70,cardiovascular/ventricular disorders/cardiomyo...,,cardiovascular,ventricular disorders,cardiomyopathy,NaN,NaN,NaN,NaN
4,141168,70,renal/disorder of kidney/chronic kidney disease,"585.9, N18.9",renal,disorder of kidney,chronic kidney disease,NaN,NaN,NaN,NaN


In [33]:
frequency1 = pd.DataFrame()
frequency1['noAnnotations'] = df1['first'].value_counts() 
frequency1['%'] = df1['first'].value_counts() / len(df1.index) * 100
frequency1

,noAnnotations,%
first,,
cardiovascular,348062,24.980479
pulmonary,241767,17.351665
neurologic,181764,13.045238
renal,165398,11.870647
gastrointestinal,107816,7.737976
endocrine,101807,7.306709
infectious diseases,83485,5.991735
hematology,67168,4.820661
burns,49800,3.574156


## Print desease trees

In [41]:
from bigtree import Node, list_to_tree, tree_to_dot, dataframe_to_tree, tree_to_pillow

for disease in frequency1.index.get_level_values('first'):
    df_ = df1[df1['first'] == disease].copy()
    df_.set_index(['patientunitstayid', 'diagnosisoffset'], inplace = True)
    df_['diagnosis'] = df_['first'] + '/' + df_['second'] + '/' + df_['third']
    
    paths_mult_ = df_.groupby('diagnosis', dropna = True)['diagnosis'].size().to_frame('noAnnotations')
    paths_mult_.reset_index(inplace=True)
    paths_mult_.sort_values('noAnnotations', ascending = False, inplace = True)

    paths_ = list(pd.unique(df_['diagnosis']))
    root_ = dataframe_to_tree(paths_mult_, sep = '/')
    root_.show(attr_list=["noAnnotations"])
    pillow_image_ = tree_to_pillow(root_)

cardiovascular
├── shock 
│   └──  hypotension [noAnnotations=103207]
├── chest pain 
│   └──  ASHD [noAnnotations=62269]
├── ventricular disorders
│   ├── congestive heart failure [noAnnotations=30103]
│   ├── hypertension [noAnnotations=16160]
│   ├── cardiomyopathy [noAnnotations=6522]
│   ├── acute pulmonary edema [noAnnotations=5339]
│   ├── low cardiac output state [noAnnotations=634]
│   ├── cardiac mural thrombus [noAnnotations=176]
│   ├── cardiac injury, blunt [noAnnotations=4]
│   └── cardiac injury, penetrating [noAnnotations=2]
├── arrhythmias
│   ├── atrial fibrillation [noAnnotations=29102]
│   ├── sinus tachycardia [noAnnotations=6582]
│   ├── bradycardia [noAnnotations=6471]
│   ├── ventricular tachycardia [noAnnotations=3915]
│   ├── AV block [noAnnotations=2518]
│   ├── atrial flutter [noAnnotations=2189]
│   ├── SVT [noAnnotations=1930]
│   ├── syncope [noAnnotations=1439]
│   ├── ventricular fibrillation [noAnnotations=729]
│   ├── pacemaker [noAnnotations=350]
│  

## Cardiovascular


In [46]:
cardio = df1[df1['first'] == 'cardiovascular']
cardio.set_index(['patientunitstayid', 'diagnosisoffset'], inplace = True)

In [47]:

cardiofreq = pd.DataFrame()
cardiofreq['noAnnotations'] = cardio['second'].value_counts() 
cardiofreq['%'] = round(cardio['second'].value_counts() / len(cardio.index) * 100, 2)
cardiofreq

,noAnnotations,%
second,,
shock,103207,29.65
chest pain,62269,17.89
ventricular disorders,58940,16.93
arrhythmias,55732,16.01
vascular disorders,36662,10.53
cardiac arrest,18758,5.39
valvular disease,3626,1.04
cardiac surgery,2799,0.80
diseases of the aorta,2306,0.66


In [48]:
cardiofreq1 = pd.DataFrame()
cardiofreq1['noAnnotations'] = cardio['third'].value_counts() 
cardiofreq1['%'] = round(cardio['third'].value_counts() / len(cardio.index) * 100, 2)
cardiofreq1

,noAnnotations,%
third,,
hypotension,103207,29.65
ASHD,62269,17.89
hypertension,44626,12.82
congestive heart failure,30103,8.65
atrial fibrillation,29102,8.36
cardiac arrest,18758,5.39
sinus tachycardia,6582,1.89
cardiomyopathy,6522,1.87
bradycardia,6471,1.86
